In [10]:
# primal barrier method from nocedal 19.6
using LinearAlgebra


function barrier(P, ∇P, x, c; μ=1, ϵ=1e-3) 
    τ = 1
    while true
        
        x, μ = steepest_descent_barrier(P, ∇P, x, μ, c, ϵ=τ)

        if norm(∇P(x, μ)) < ϵ
            break
        end

        τ = τ * 0.9
        μ = μ * 0.9      
    end
    return x, μ
end


function backtracking_line_search_barrier(f, ∇f, x, μ, c, p)
    α = .5     # initial step length
    rho = 0.75
    ctol = 1e-4

    function stay_feasible(α)
        while c(x + α*p) < 0
            α *= rho
        end
        return α
    end
    α = stay_feasible(α)

    while f(x + α*p, μ) > f(x, μ) + ctol*α*transpose(∇f(x, μ))*p 
        α *= rho
        α = stay_feasible(α)
    end
    return α
end


# steepest descent modified to be used with a barrier method
function steepest_descent_barrier(P, ∇P, x, μ, c; ϵ=1e-3, k=1, c1=1e-4)

    i = 1
    while norm(∇P(x, μ)) > ϵ

        p = -∇P(x, μ)
    
        α = backtracking_line_search_barrier(P, ∇P, x, μ, c, p)

        x = x + α*p  
        i += 1
    end
    return x, μ
end


# min (x1 + 0.5)^2 + (x2 - 0.5)^2
# s.t. x1 >= 0
f(x) = (x[1]+0.5)^2 + (x[2]-0.5)^2
c(x) = x[1]
B(x) = -log(c(x))
P(x,μ) = f(x) + μ*B(x)
∇P(x,μ) = [ 2*(x[1]+0.5)-μ/x[1], 2*(x[2]-0.5) ]  

x0 = [1,1]
(x,μ) = barrier(P, ∇P, x0, c; μ=1.0)
println("solution occurs near x = ", x)

solution occurs near x = [0.0013076864951298472, 0.5000000000000041]


I created two functions here, steepest_descent_barrier and barrier. Its the basic steepest_descent algorithm. Barrier follows algorithm 19.5 as highlighted in Nocedal. It continuously sets epsilon to equal tau, which is always decreasing with each iteration (tau = tau * .9), and breaks once the norm of the gradient is less than epsilon. 